In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/tviel/work/kaggle_rsna_lumbar_spine/src


/home/tviel/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
import cv2
import json
import glob
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from collections import Counter
from tqdm.notebook import tqdm


pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [4]:
from params import *

from data.preparation import prepare_data

from util.plots import *
from util.logger import Config
from util.torch import load_model_weights

from model_zoo.models_seg import define_model

In [5]:
df = prepare_data(DATA_PATH)

In [6]:
df_train = pd.read_csv(DATA_PATH + "train.csv")

In [16]:
df_train[df_train[df_train.columns[:6]].isna().sum(1) > 0]

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1378,3008676218,NaN,NaN,NaN,NaN,NaN,Normal/Mild,Severe,Moderate,Moderate,...,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Severe,Normal/Mild,Normal/Mild


In [21]:
len(df_train), len(df.study_id.unique())

(1975, 1975)

In [22]:
import re
df['condition'] = df['condition'].apply(lambda d: d if isinstance(d, list) else [])

df['condition'] = df['condition'].apply(lambda x: [re.sub('Neural Foraminal Narrowing', 'NFN', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Spinal Canal Stenosis', 'SCS', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Subarticular Stenosis', 'SS', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Left', 'L', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Right', 'R', u) for u in x])

In [23]:
df[df['weighting'] == "T1"].condition.apply(len).value_counts() / (len(df) / 3)

condition
10    0.935653
5     0.006673
0     0.000477
15    0.000477
9     0.000477
Name: count, dtype: float64

In [24]:
df[df['weighting'] == "T2/STIR"].condition.apply(len).value_counts()

condition
5    1898
3      39
4      35
1       1
0       1
Name: count, dtype: int64

In [25]:
df_ = pd.read_csv(DATA_PATH + "train_label_coordinates.csv")
df_ = df_[df_['condition'].apply(lambda x: 'Neural Foraminal Narrowing' in x)].reset_index(drop=True)
df_["side"] = df_['condition'].apply(lambda x: x.split()[0])

frames = pd.read_csv('../input/df_frames.csv')
frames["frames"] = frames["frames"].apply(eval)
df_ = df_.merge(frames, how="left")
df_['ordered'] = df_['frames'].apply(lambda x: x[0] < x[-1])

dfg = df_[["study_id", "series_id", "side", "instance_number", "ordered"]].groupby(["study_id", "series_id", "side"]).mean()
dfg['ordered'] = dfg['ordered'] > 0.5
dfg = dfg.reset_index().groupby(["study_id", "series_id"])

In [26]:
df['x'] = df['coords'].apply(lambda x: [int(u[0]) for u in x] if isinstance(x, np.ndarray) else -1)

In [29]:
df_train

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild


In [27]:
df

,study_id,series_id,series_description,orient,weighting,img_path,condition,level,coords,x
0,4003253,702807833,Sagittal T2/STIR,Sagittal,T2/STIR,../input/npy/4003253_702807833.npy,"[SCS, SCS, SCS, SCS, SCS]","[L1/L2, L2/L3, L3/L4, L4/L5, L5/S1]","[[7.0, 322.83185840707966, 227.9646017699115], [7.0, 320.57142857142856, 295.7142857142857], [7....","[7, 7, 7, 7, 7]"
1,4003253,1054713880,Sagittal T1,Sagittal,T1,../input/npy/4003253_1054713880.npy,"[R NFN, R NFN, R NFN, R NFN, R NFN, L NFN, L NFN, L NFN, L NFN, L NFN]","[L4/L5, L5/S1, L3/L4, L1/L2, L2/L3, L1/L2, L4/L5, L5/S1, L2/L3, L3/L4]","[[3.0, 187.96175908221795, 251.83938814531547], [3.0, 198.2409177820268, 285.6137667304015], [4....","[3, 3, 4, 5, 5, 10, 10, 10, 11, 11]"
2,4003253,2448190387,Axial T2,Axial,T2,../input/npy/4003253_2448190387.npy,"[L SS, R SS, L SS, R SS, L SS, R SS, L SS, R SS, L SS, R SS]","[L1/L2, L1/L2, L2/L3, L2/L3, L3/L4, L3/L4, L4/L5, L4/L5, L5/S1, L5/S1]","[[2.0, 179.12644787644788, 161.23552123552125], [3.0, 145.28877148997134, 158.6246418338109], [1...","[2, 3, 10, 10, 18, 18, 27, 27, 34, 34]"
3,4646740,3201256954,Axial T2,Axial,T2,../input/npy/4646740_3201256954.npy,"[R SS, L SS, L SS, R SS, R SS, L SS, L SS, R SS, L SS, R SS]","[L1/L2, L1/L2, L2/L3, L2/L3, L3/L4, L3/L4, L4/L5, L4/L5, L5/S1, L5/S1]","[[14.0, 184.18099547511312, 263.23981900452486], [15.0, 235.3170731707317, 264.08362369337976], ...","[14, 15, 21, 21, 27, 28, 33, 33, 39, 39]"
4,4646740,3486248476,Sagittal T1,Sagittal,T1,../input/npy/4646740_3486248476.npy,"[L NFN, L NFN, L NFN, L NFN, L NFN, R NFN, R NFN, R NFN, R NFN, R NFN]","[L2/L3, L3/L4, L4/L5, L1/L2, L5/S1, L2/L3, L3/L4, L1/L2, L4/L5, L5/S1]","[[12.0, 234.09931142986449, 196.620230591671], [12.0, 227.5139455762059, 256.8292898251205], [12...","[12, 12, 12, 11, 10, 2, 2, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...
6289,4287160193,1507070277,Sagittal T2/STIR,Sagittal,T2/STIR,../input/npy/4287160193_1507070277.npy,"[SCS, SCS, SCS, SCS, SCS]","[L1/L2, L2/L3, L3/L4, L4/L5, L5/S1]","[[7.0, 391.2351904090268, 235.6445698166432], [7.0, 369.4354724964739, 321.80535966149506], [7.0...","[7, 7, 7, 7, 7]"
6290,4287160193,1820446240,Axial T2,Axial,T2,../input/npy/4287160193_1820446240.npy,"[L SS, R SS, L SS, R SS, L SS, R SS, L SS, R SS, L SS, R SS]","[L5/S1, L5/S1, L4/L5, L4/L5, L3/L4, L3/L4, L2/L3, L2/L3, L1/L2, L1/L2]","[[38.0, 145.7037037037037, 143.40740740740742], [38.0, 112.16991150442476, 143.34159292035395], ...","[38, 38, 33, 32, 26, 26, 20, 20, 14, 14]"
6291,4290709089,3274612423,Sagittal T2/STIR,Sagittal,T2/STIR,../input/npy/4290709089_3274612423.npy,"[SCS, SCS, SCS, SCS, SCS]","[L1/L2, L2/L3, L3/L4, L4/L5, L5/S1]","[[8.0, 181.66894664842684, 88.86456908344734], [8.0, 174.22708618331055, 125.19835841313272], [8...","[8, 8, 8, 8, 8]"
6292,4290709089,3390218084,Axial T2,Axial,T2,../input/npy/4290709089_3390218084.npy,"[R SS, L SS, L SS, R SS, L SS, R SS, L SS, R SS, L SS, R SS]","[L1/L2, L1/L2, L2/L3, L2/L3, L3/L4, L3/L4, L4/L5, L4/L5, L5/S1, L5/S1]","[[1.0, 307.18084360986546, 354.8699595361547], [2.0, 352.53169907016064, 358.1403212172443], [4....","[1, 2, 4, 5, 9, 9, 14, 14, 19, 20]"


In [31]:
df[df['study_id'] == 3637444890]

,study_id,series_id,series_description,orient,weighting,img_path,condition,level,coords,x
5330,3637444890,1590743005,Axial T2,Axial,T2,../input/npy/3637444890_1590743005.npy,"[L SS, R SS, L SS, R SS, L SS, R SS]","[L3/L4, L3/L4, L4/L5, L4/L5, L5/S1, L5/S1]","[[3.0, 176.13040629095676, 175.72739187418088], [3.0, 153.80185979971387, 173.5050071530758], [1...","[3, 3, 10, 11, 16, 17]"
5331,3637444890,3671685092,Axial T2,Axial,T2,../input/npy/3637444890_3671685092.npy,"[L SS, R SS, L SS, R SS]","[L1/L2, L1/L2, L2/L3, L2/L3]","[[9.0, 183.26015727391876, 175.72739187418088], [9.0, 147.85050071530756, 174.87839771101574], [...","[9, 9, 17, 17]"
5332,3637444890,3892989905,Sagittal T2/STIR,Sagittal,T2/STIR,../input/npy/3637444890_3892989905.npy,[],NaN,NaN,-1
5333,3637444890,3951475160,Sagittal T1,Sagittal,T1,../input/npy/3637444890_3951475160.npy,"[R NFN, R NFN, R NFN, R NFN, R NFN, SCS, SCS, SCS, SCS, SCS, L NFN, L NFN, L NFN, L NFN, L NFN]","[L4/L5, L5/S1, L1/L2, L2/L3, L3/L4, L1/L2, L2/L3, L3/L4, L4/L5, L5/S1, L1/L2, L2/L3, L3/L4, L4/L...","[[3.0, 194.8471074380165, 224.26446280991732], [3.0, 201.72314049586777, 259.702479338843], [4.0...","[3, 3, 4, 4, 4, 7, 7, 7, 7, 7, 10, 11, 11, 11, 11]"


In [33]:
df[df['study_id'] == 3637444890]

,study_id,series_id,series_description,orient,weighting,img_path,condition,level,coords,x
5330,3637444890,1590743005,Axial T2,Axial,T2,../input/npy/3637444890_1590743005.npy,"[L SS, R SS, L SS, R SS, L SS, R SS]","[L3/L4, L3/L4, L4/L5, L4/L5, L5/S1, L5/S1]","[[3.0, 176.13040629095676, 175.72739187418088], [3.0, 153.80185979971387, 173.5050071530758], [1...","[3, 3, 10, 11, 16, 17]"
5331,3637444890,3671685092,Axial T2,Axial,T2,../input/npy/3637444890_3671685092.npy,"[L SS, R SS, L SS, R SS]","[L1/L2, L1/L2, L2/L3, L2/L3]","[[9.0, 183.26015727391876, 175.72739187418088], [9.0, 147.85050071530756, 174.87839771101574], [...","[9, 9, 17, 17]"
5332,3637444890,3892989905,Sagittal T2/STIR,Sagittal,T2/STIR,../input/npy/3637444890_3892989905.npy,[],NaN,NaN,-1
5333,3637444890,3951475160,Sagittal T1,Sagittal,T1,../input/npy/3637444890_3951475160.npy,"[R NFN, R NFN, R NFN, R NFN, R NFN, SCS, SCS, SCS, SCS, SCS, L NFN, L NFN, L NFN, L NFN, L NFN]","[L4/L5, L5/S1, L1/L2, L2/L3, L3/L4, L1/L2, L2/L3, L3/L4, L4/L5, L5/S1, L1/L2, L2/L3, L3/L4, L4/L...","[[3.0, 194.8471074380165, 224.26446280991732], [3.0, 201.72314049586777, 259.702479338843], [4.0...","[3, 3, 4, 4, 4, 7, 7, 7, 7, 7, 10, 11, 11, 11, 11]"


In [110]:
for s in tqdm(df.series_id.unique()):
    dfs = df[df['series_id'] == s]

    if dfs["weighting"].values[0] != "T1":
        continue

    xs = dfs['x'].values[0]
    sides = [l[0] for l in dfs['condition'].values[0]]

    dfc = pd.DataFrame({'x': xs, 's': sides})
    dfc = dfc.groupby("s").mean()
    
    try:
        if not dfc["x"]["R"] < dfc["x"]["L"]:
            print(s)
    except KeyError:
        pass
    # display(dfc)
    # break

  0%|          | 0/6294 [00:00<?, ?it/s]

1518511736
2278678071
3230157587
1032434193
692927423
3802667261
856763877


In [108]:
df[df['series_id'] == 1518511736]

,study_id,series_id,series_description,orient,weighting,img_path,condition,level,coords,x
1561,1085426528,1518511736,Sagittal T1,Sagittal,T1,../input/npy/1085426528_1518511736.npy,"[R NFN, R NFN, R NFN, R NFN, R NFN, L NFN, L NFN, L NFN, L NFN, L NFN]","[L2/L3, L3/L4, L1/L2, L4/L5, L5/S1, L3/L4, L1/L2, L2/L3, L4/L5, L5/S1]","[[10.0, 162.7428303524633, 154.96855345911948], [10.0, 159.08407079646017, 190.01769911504425], ...","[10, 10, 9, 8, 8, 4, 3, 3, 3, 1]"


In [14]:
df_train[df_train['study_id'] == 4003253].T

,0
study_id,4003253
spinal_canal_stenosis_l1_l2,Normal/Mild
spinal_canal_stenosis_l2_l3,Normal/Mild
spinal_canal_stenosis_l3_l4,Normal/Mild
spinal_canal_stenosis_l4_l5,Normal/Mild
spinal_canal_stenosis_l5_s1,Normal/Mild
left_neural_foraminal_narrowing_l1_l2,Normal/Mild
left_neural_foraminal_narrowing_l2_l3,Normal/Mild
left_neural_foraminal_narrowing_l3_l4,Normal/Mild
left_neural_foraminal_narrowing_l4_l5,Moderate


In [8]:
df_train

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild


Done ! 